In [1]:
%load_ext autoreload
%autoreload 2

In this lab, we are going to build an interactive chatbot using a modular, object-oriented architecture designed around LLM (Large Language Model) utilities. The goal is not just to create a working chatbot, but to demonstrate how object-oriented design patterns, SOLID principles, and software architecture best practices can be applied in a real-world AI system. We’ll start by examining the UML class diagram, which outlines the key components of our system—such as the LLMFactory, the different LLMBuilder implementations, and the ConversationManager—and how they interact. Throughout the session, you’ll see how patterns like Factory, Builder, and Strategy work together to make the chatbot extensible and maintainable, while principles like Single Responsibility and Dependency Inversion ensure clean separation of concerns. By the end of the lab, you’ll understand not only how to instantiate and configure different language models (like OpenAI, LM Studio, or HuggingFace) through a unified interface, but also how thoughtful design choices lead to flexible and scalable software systems.

## Lab Objectives
- Apply key software design patterns (Factory, Builder, Strategy) in the context of AI applications
- Recognize how design principles (SOLID, DRY) improve code maintainability and flexibility
- Analyze the flow of data and control in a component-based architecture
- Apply event-driven design concepts to manage conversation state

## Architectural notes:
The architecture is layered because the system is organized into distinct levels of responsibility — from model creation (via factories and builders) to orchestration (conversation management) to execution (model invocation) and finally to the user interface. Each layer depends only on the one below it, promoting clear separation of concerns and easier maintainability.

It is partially event-driven because the conversation flow — handled internally through LangGraph — reacts to state changes and transitions between nodes based on events, rather than following a strictly linear, procedural sequence. This event-driven behavior is localized within the conversation management layer, while the overall system structure remains layered.

```mermaid
classDiagram
    %% Core Factory Implementation
    class LLMFactory {
        -Dict[str, LLMBuilder] _builders
        +register_builder(name: str, builder: LLMBuilder) void
        +get_builder(name: str) LLMBuilder
        +create_from_yaml_file(config_path: Path) BaseChatModel
        -_load_config(config_path: Path) Dict[str, Any]
    }

    %% Builder Pattern Implementation
    class LLMBuilder {
        <<abstract>>
        +build(**kwargs) BaseChatModel
    }

    class OpenAILLMBuilder {
        +build(model_name: str, api_key: str, **kwargs) ChatOpenAI
        -_get_api_key_from_env(force_reload: bool) str
    }

    class LMStudioBuilder {
        +build(model_name: str, base_url: str, **kwargs) ChatOpenAI
    }

    class HuggingFaceBuilder {
        +build(model_name: str, **kwargs) HuggingFacePipeline
    }

    %% Conversation Management
    class ConversationManager {
        -BaseChatModel llm
        -bool verbose
        -str thread_id
        -Dict config
        -StateGraph graph
        -Dict config_dict
        +__init__(llm: BaseChatModel, config_path: Path, verbose: bool, thread_id: str)
        +chat(message: str) str
        +get_history() List[Dict[str, str]]
        +reset() void
        -_load_config(config_path: Path) void
        -_build_graph() StateGraph
        +from_yaml(llm: BaseChatModel, config_path: Path, verbose: bool, thread_id: str) ConversationManager
    }

    %% External LangChain Classes
    class BaseChatModel {
        <<interface>>
        +invoke(messages: List) AIMessage
    }

    class StateGraph {
        +add_node(name: str, func) void
        +add_edge(source, target) void
        +compile(checkpointer) Graph
        +invoke(inputs: Dict, config: Dict) Dict
    }

    %% Language Model Implementations
    class ChatOpenAI {
        +invoke(messages: List) AIMessage
    }

    class HuggingFacePipeline {
        +invoke(messages: List) AIMessage
    }

    %% Relationships
    LLMBuilder <|-- OpenAILLMBuilder
    LLMBuilder <|-- LMStudioBuilder
    LLMBuilder <|-- HuggingFaceBuilder
    
    LLMFactory --> LLMBuilder : uses >
    LLMFactory ..> BaseChatModel : creates >
    
    OpenAILLMBuilder ..> ChatOpenAI : creates >
    LMStudioBuilder ..> ChatOpenAI : creates >
    HuggingFaceBuilder ..> HuggingFacePipeline : creates >
    
    ChatOpenAI --|> BaseChatModel : implements
    HuggingFacePipeline --|> BaseChatModel : implements
    
    ConversationManager o-- BaseChatModel : contains >
    ConversationManager --> StateGraph : creates >
    
    %% Interactive Chat
    class InteractiveChat {
        +main() void
    }
    
    InteractiveChat --> LLMFactory : uses >
    InteractiveChat --> ConversationManager : uses >
```

To better understand how the components interact in this architecture, let's follow the flow of what happens when a user wants to chat with an LMStudio-based local LLM:

1. **Initialization Phase**:
   - The application starts by loading configuration from YAML files:
     - `config.yaml` contains model configurations for different LLM providers
     - `conversation.yaml` defines memory and conversation parameters
   
   - `LLMFactory` is instantiated to create the appropriate LLM implementation
   - The factory identifies "lmstudio" as the requested model type
   - It retrieves the `LMStudioBuilder` from its registry of builders

2. **Model Building Phase**:
   - `LMStudioBuilder` is activated with configuration parameters:
     - `model_name: "qwen2.5-7b-instruct-1m"`
     - `base_url: "http://127.0.0.1:1234/v1"`
     - `temperature: 0.0`
   
   - The builder validates the connection to the local LM Studio server
   - It constructs a `ChatOpenAI` instance configured to communicate with the local server
   - The builder returns this instance to the factory, which returns it to the client

3. **Conversation Setup Phase**:
   - `ConversationManager` is created with the LLM instance
   - It loads conversation configuration (memory settings, etc.)
   - It builds a LangGraph state graph for managing conversation flow
   - Each node in the graph represents a processing stage for messages

4. **Interaction Phase**:
   - User sends a message via `chat(message)`
   - The message is wrapped as a `HumanMessage` object
   - The message enters the state graph as part of the conversation state
   - The graph executes the `call_model` node, which:
     - Applies memory management (trimming based on configured strategy)
     - Invokes the LLM with the current message context
     - Captures the response and adds it to the conversation state
   - The response is extracted and returned to the user

5. **Memory Management**:
   - Throughout this process, the `ConversationManager` maintains conversation history
   - Messages are trimmed according to the configured strategy (e.g., keeping last N messages)
   - The conversation state is preserved for subsequent interactions
   - Users can retrieve the full conversation history or reset it as needed

In [2]:
import yaml
from pathlib import Path

from im2203.llm_utils.factory import LLMFactory
from im2203.llm_utils.chat import ConversationManager


d:\conda\envs\im2203\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Create the LLM factory
llm_factory = LLMFactory()

# Load LM Studio config
lms_config = Path("../configs/config.yaml")
chat_config = Path("../configs/conversation.yaml")



In [6]:
import json
# dump the contents of lms_config:
with open(lms_config, "r") as f:
    lms_config_dict = yaml.safe_load(f)

# print the config dict in json nested format
print(json.dumps(lms_config_dict, indent=2))

{
  "default_model": "openai",
  "models": {
    "openai": {
      "name": "openai",
      "model_name": "gpt-3.5-turbo",
      "base_url": null,
      "temperature": 0.0
    },
    "lmstudio": {
      "name": "lmstudio",
      "model_name": "qwen2.5-7b-instruct-1m",
      "base_url": "http://127.0.0.1:1234/v1",
      "temperature": 0.0
    },
    "huggingface": {
      "name": "huggingface",
      "model_name": "meta-llama/Llama-2-7b-chat-hf",
      "temperature": 0.3,
      "quantize_4bit": false,
      "max_new_tokens": 500,
      "top_k": 50,
      "top_p": 0.95
    }
  }
}


In [7]:
llm = llm_factory.create_from_yaml_file(lms_config)


D:\Repos2\.env
Loading API key from: D:\Repos2\.env
Using API key: sk-pr...JXMkA


In [8]:
chat = ConversationManager(llm=llm, config_path=chat_config, verbose=True)

In [9]:
chat.chat("message=1")
chat.chat("message=2")
chat.chat("message=3")
chat.chat("message=4")

🔍 Trimmed to 1 messages
  - HumanMessage: message=1
🔍 Trimmed to 3 messages
  - HumanMessage: message=1
  - AIMessage: Hello! How can I assist you today?
  - HumanMessage: message=2
🔍 Trimmed to 3 messages
  - HumanMessage: message=2
  - AIMessage: I'm here to help! What do you need assistance with?
  - HumanMessage: message=3
🔍 Trimmed to 3 messages
  - HumanMessage: message=3
  - AIMessage: How can I assist you today?
  - HumanMessage: message=4


"I'm here to help! What do you need assistance with?"

In [10]:
chat.get_history()

[{'role': 'user', 'content': 'message=1'},
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'},
 {'role': 'user', 'content': 'message=2'},
 {'role': 'assistant',
  'content': "I'm here to help! What do you need assistance with?"},
 {'role': 'user', 'content': 'message=3'},
 {'role': 'assistant', 'content': 'How can I assist you today?'},
 {'role': 'user', 'content': 'message=4'},
 {'role': 'assistant',
  'content': "I'm here to help! What do you need assistance with?"}]

Now that everything is working let's run the interactive chat from the terminal